In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import warnings
warnings.filterwarnings('ignore')
from corredores import infos, corrida
import sqlalchemy as db

# Extraindo dados direto da API

In [2]:
url_utmb_brazil = 'https://api.utmb.world/search/runners?category=general&sex=&ageGroup=&nationality=BR&limit=1000000&offset=0&search='

runners = requests.get(url_utmb_brazil)
runners_br_df = pd.DataFrame(runners.json() ["runners"])
runners_br_df = runners_br_df.drop(columns=['picture'])


In [3]:
runners_br_df['fullname'].to_csv('runners_br_names.csv', index=False)
runners_br_df.to_csv('runners_br.csv', index=False)
runners_br_df

,id,ageGroup,fullname,uri,ip,nationality,sex
0,1359763,35-39,Rogerio SILVESTRIN,1359763.rogerio.silvestrin,854,BR,H
1,2623403,35-39,Wellington ROBERTO NORONHA,2623403.wellington.robertonoronha,828,BR,H
2,395551,45-49,Celio AUGUSTO ROSA,395551.celio.augustorosa,825,BR,H
3,5027328,20-34,Johnny LUNA-LIMA,5027328.johnny.lunalima,814,BR,H
4,5268870,45-49,Cordeiro Valdenir JANDOSA,5268870.cordeirovaldenir.jandosa,807,BR,H
...,...,...,...,...,...,...,...
36838,5932185,45-49,Fabio AZEVEDO,5932185.fabio.azevedo,0,BR,H
36839,5935787,35-39,CAMILA FARIA,5935787.camila.faria,0,BR,F
36840,5935803,20-34,Alexandre FERREIRA,5935803.alexandre.ferreira,0,BR,H
36841,5938287,50-54,Wesley ALMEIDA,5938287.wesley.almeida,0,BR,H


In [4]:
dic = runners.json()["runners"][0]
del dic['picture']
del dic['ip']
dic.update(infos(runners_br_df['uri'][0]))
dic

{'id': 1359763,
 'ageGroup': '35-39',
 'fullname': 'Rogerio SILVESTRIN',
 'uri': '1359763.rogerio.silvestrin',
 'nationality': 'BR',
 'sex': 'H',
 'Club': 'Go On Outdoor',
 'Team': 'Go On Outdoor',
 'Sponsor(s)': 'Go On Outdoor',
 'Description': None,
 'Geral': '854',
 '20K': None,
 '50K': '853',
 '100K': '826',
 '100m': None}

In [5]:
engine = db.create_engine('sqlite:///utmb.sqlite', connect_args={'timeout': 15})
connection = engine.connect()
metadata = db.MetaData()

In [6]:
corredores = db.Table('corredores', metadata,
                        db.Column('id', db.Integer(), primary_key=True),
                        db.Column('ageGroup', db.String(255)),
                        db.Column('fullname', db.String(255)),
                        db.Column('uri', db.String(255)),
                        db.Column('nationality', db.String(255)),
                        db.Column('sex', db.String(255)),
                        db.Column('Club', db.String(255)),
                        db.Column('Team', db.String(255)),
                        db.Column('Sponsor(s)', db.String(255)),
                        db.Column('Description', db.String(255)),
                        db.Column('Geral', db.String(255)),
                        db.Column('20K', db.String(255)),
                        db.Column('50K', db.String(255)),
                        db.Column('100K', db.String(255)),
                        db.Column('100m', db.String(255)))

metadata.create_all(engine)

In [7]:
query = db.insert(corredores)
ResultProxy = connection.execute(query)

In [8]:
dic

# Inserir dic no banco de dados 'corredores'

{'id': 1359763,
 'ageGroup': '35-39',
 'fullname': 'Rogerio SILVESTRIN',
 'uri': '1359763.rogerio.silvestrin',
 'nationality': 'BR',
 'sex': 'H',
 'Club': 'Go On Outdoor',
 'Team': 'Go On Outdoor',
 'Sponsor(s)': 'Go On Outdoor',
 'Description': None,
 'Geral': '854',
 '20K': None,
 '50K': '853',
 '100K': '826',
 '100m': None}

In [2]:
from corredores import corrida, corredor_corrida

print(corrida('329.hardrock100endurancerunhardrock100-cw.2000'))
print('-'*40)
print(len(corredor_corrida('329.hardrock100endurancerunhardrock100-cw.2000')))

{'Title': 'HARDROCK 100 ENDURANCE RUN 2000 - HARDROCK 100 - CW', 'City / Country': 'United States', 'Day': '7', 'Month': 'July', 'Year': '2000', 'Race Category': '100M', 'Distance': '161.8', 'Elevation Gain': '10365'}
----------------------------------------
60
